In [ ]:
import requests, json, os
os.chdir('/fsx/home/hermee/projects/python/chem-agent')

BASE = 'http://localhost:8000/api'

# Health check
r = requests.get(f'{BASE}/health')
print('Health:', r.json())

In [ ]:
# List reactions
r = requests.get(f'{BASE}/reactions')
for rxn in r.json()['reactions']:
    warn = f" ⚠️ {rxn['warning']}" if rxn.get('warning') else ''
    print(f"  {rxn['id']}: {rxn['name']} ({rxn['reactants']}){warn}")

In [ ]:
# Chat via SSE streaming
question = 'Design a 3-tail ionizable lipid with amine heads. Which reaction templates should I use?'

response = requests.post(f'{BASE}/chat', json={'message': question}, stream=True)
for line in response.iter_lines():
    if not line:
        continue
    line = line.decode('utf-8')
    if not line.startswith('data: '):
        continue
    data = line[6:]
    if data == '[DONE]':
        print('\n✅ Done')
        break
    parsed = json.loads(data)
    if parsed['type'] == 'status':
        print(f"  {parsed['message']}")
    elif parsed['type'] == 'answer':
        print(f"\n📋 ANSWER:\n{parsed['content'][:2000]}")

In [ ]:
# Full query (non-streaming, returns all expert analyses)
r = requests.post(f'{BASE}/query', json={'query': 'What are the issues with reaction 10012 and 10017?'})
result = r.json()

for key in ['reaction_analysis', 'design_rules_check', 'synthesis_plan', 'final_answer']:
    print(f'\n{"=" * 50}')
    print(f'{key.upper()}')
    print(f'{"=" * 50}')
    print(result[key][:1000])